In [ ]:
import numpy as np
import pyomo.environ as pe
import pyomo.opt
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as mcolors
from pyomo.util.infeasible import log_infeasible_constraints
import os
import pandas as pd
import dotenv
from new_model import DC_Community_Opt

# DC Community

In [ ]:
# Set default behaviour
default_behaviour = pe.Constraint.Skip

In [ ]:
# Auxiliary function to convert numpy arrays to dictionaries
def convert_to_dictionary(a, t_start=0):
    temp_dictionary = {}

    if len(a.shape) == 3:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                for dim2 in np.arange(a.shape[2]):
                    temp_dictionary[(dim0+1, dim1+1, dim2+1+t_start)] = a[dim0, dim1, dim2]
    elif len(a.shape) == 2:
        for dim0 in np.arange(a.shape[0]):
            for dim1 in np.arange(a.shape[1]):
                temp_dictionary[(dim0+1, dim1+1+t_start)] = a[dim0, dim1]

    else:
        for dim0 in np.arange(a.shape[0]):
            temp_dictionary[(dim0+1+t_start)] = a[dim0]

    return temp_dictionary

In [ ]:
conf = dotenv.dotenv_values()
executable = conf["PATH_TO_SOLVER"]

In [ ]:
excel_file = './grid_data_input_file_building_demo.xlsx'
dc_community_opt = DC_Community_Opt(excel_file)
dc_community_opt.execute(executable)

In [ ]:
def ext_pyomo_vals(vals):
    # make a pd.Series from each
    s = pd.Series(vals.extract_values(),
                  index=vals.extract_values().keys())

    # if the series is multi-indexed we need to unstack it...
    if type(s.index[0]) == tuple:    # it is multi-indexed
        s = s.unstack(level=1)
    else:
        # force transition from Series -> df
        s = pd.DataFrame(s)

    return s

In [ ]:
result_genActPower = ext_pyomo_vals(dc_community_opt.model.genActPower)
result_genActPower = result_genActPower.T
result_genActPower.columns = ["genActPower"+ f'{g}' for g in np.arange(1, dc_community_opt.model.gen.last() + 1)]

result_genExcPower = ext_pyomo_vals(dc_community_opt.model.genExcPower)
result_genExcPower = result_genExcPower.T
result_genExcPower.columns = ["genExcPower"+ f'{g}' for g in np.arange(1, dc_community_opt.model.gen.last() + 1)]

result_pimp= ext_pyomo_vals(dc_community_opt.model.imports)
result_pimp.columns = ["grid_import"]

result_pexp = ext_pyomo_vals(dc_community_opt.model.exports)
result_pexp.columns = ["grid_export"]

result_P_import_relax = ext_pyomo_vals(dc_community_opt.model.P_import_relax)
result_P_import_relax.columns = ["P_import_relax"]

if len(dc_community_opt.model.stor) > 0:
    result_storCharge = ext_pyomo_vals(dc_community_opt.model.storCharge)
    result_storCharge = result_storCharge.T
    result_storCharge.columns = ["storCharge"+ f'{s}' for s in np.arange(1, model.stor.last() + 1)]

    result_storDischarge = ext_pyomo_vals(dc_community_opt.model.storDischarge)
    result_storDischarge = result_storDischarge.T
    result_storDischarge.columns = ["storDischarge"+ f'{s}' for s in np.arange(1, dc_community_opt.model.stor.last() + 1)]

    result_storState = ext_pyomo_vals(dc_community_opt.model.storState)
    result_storState = result_storState.T
    result_storState.columns = ["storState"+ f'{s}' for s in np.arange(1, dc_community_opt.model.stor.last() + 1)]

result_v2gCharge = ext_pyomo_vals(dc_community_opt.model.v2gCharge)
result_v2gCharge = result_v2gCharge.T
result_v2gCharge.columns = ["v2gCharge"+ f'{v}' for v in np.arange(1, dc_community_opt.model.v2g.last() + 1)]

result_v2gDischarge = ext_pyomo_vals(dc_community_opt.model.v2gDischarge)
result_v2gDischarge = result_v2gDischarge.T
result_v2gDischarge.columns = ["v2gDischarge"+ f'{v}' for v in np.arange(1, dc_community_opt.model.v2g.last() + 1)]

result_v2gState = ext_pyomo_vals(dc_community_opt.model.v2gState)
result_v2gState = result_v2gState.T
result_v2gState.columns = ["v2gState"+ f'{v}' for v in np.arange(1, dc_community_opt.model.v2g.last() + 1)]

result_loadValues = ext_pyomo_vals(dc_community_opt.model.loadValues)
result_loadValues = result_loadValues.T
result_loadValues.columns = ["loadValues"+ f'{l}' for l in np.arange(1, dc_community_opt.model.loads.last() + 1)]

opt_results_df = pd.DataFrame()
opt_results_df = pd.concat([result_genActPower, result_genExcPower, 
                            result_pimp, result_pexp, result_P_import_relax,
                            result_loadValues,
                            #result_storCharge, result_storDischarge,
                            #result_storState, result_v2gCharge, 
                            result_v2gDischarge, result_v2gState
                            ],
                           axis=1)

opt_results_df.to_excel("opt_results.xlsx")
        